In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial_value=initial, name=name)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial_value=initial, name=name)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [4]:
x = tf.placeholder(tf.float32, [None, 784], name='x-input')
y = tf.placeholder(tf.float32, [None, 10], name='y-input')
lr = tf.Variable(0.001, dtype = tf.float32)

x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image')

W_conv1 = weight_variable([5, 5, 1, 32], name='W_conv1')
b_conv1 = bias_variable([32], name='b_conv1')
conv2d_1 = conv2d(x_image, W_conv1) + b_conv1
h_conv1 = tf.nn.relu(conv2d_1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64], name='W_conv2')
b_conv2 = bias_variable([64], name='b_conv2')
conv2d_2 = conv2d(h_pool1, W_conv2) + b_conv2
h_conv2 = tf.nn.relu(conv2d_2)
h_pool2 = max_pool_2x2(h_conv2)

In [5]:
W_fc1 = weight_variable([7 * 7 * 64, 1000], name='W_fc1')
b_fc1 = bias_variable([1000], name='b_fc1')

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64], name='h_pool2_flat')
wx_plus_b1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(wx_plus_b1)
keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name='h_fc1_drop')

W_fc2 = weight_variable([1000, 10], name='W_fc2')
b_fc2 = bias_variable([10], name='b_fc2')

wx_plus_b2 = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
prediction = tf.nn.softmax(wx_plus_b2)

In [6]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction), name='cross_entropy')
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
with tf.device('/gpu:0'):
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        batch_size = 100
        batch = (int) (60000 / batch_size)

        for _ in range(101):
            sess.run(tf.assign(lr, 0.0001 * (0.95 ** _)))
            for batch_step  in range(batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.68})

            if _ % 20 == 0:
                test_acc = sess.run(accuracy,feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
                print("Iterator: ", _, "Accuracy:", test_acc)

Iterator:  0 Accuracy: 0.954
Iterator:  20 Accuracy: 0.9912
Iterator:  40 Accuracy: 0.9916
Iterator:  60 Accuracy: 0.9924
Iterator:  80 Accuracy: 0.9924
Iterator:  100 Accuracy: 0.9926
